# Team Introduction
Our group is comprised of Braden Anderson, Hien Lam, and Tavin Weeda.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
# from lightgbm import LGBMClassifier
from scipy import stats

from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression, SGDRegressor
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, ConfusionMatrixDisplay, RocCurveDisplay, make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif, RFECV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.base import clone
from sklearn.neighbors import KNeighborsRegressor

# Business Understanding 1
- Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). 
- How will you measure the effectiveness of a good algorithm? 
- Why does your chosen validation method make sense for this specificdataset and the stakeholders needs?  
(10)

# Data Understanding 1
- Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. 
- Verify data quality: Are there missing values? 
- Duplicate data? Outliers? Are those mistakes? 
- How do you deal with these problems?  
(10)

## Section Summary:
- words
- words
- words
- words

## Comprehensive Data Cleaning Steps
1. Read, merged, filtered the data (54427 rows, 24 columns) then dealt with missing values to result in (54473, 24)
2. Inspected each feature, noted cleaning steps if necessary (features from lab 1 only)
3. Carefully binned existing features (`body_type_binned`, `intersection_binned`, `weather_binned`)
4. Derived new features (`speeding_status`, `hour_binned`, `restraint_binned`, `pcrash1_imname`)
5. Binned and visualized response variable for classification task, `maxsev_binned`
6. Visualized response variable for regression task, `age_im`
7. Confirmed there are zero duplicates and NA values (50535, 24)
8. Conducted data preprocessing: 
  - Split full dataset into 80% train / 20% test, performed 10 fold stratified cross validation within the 80% train set.
  - Scaled `age_im` and `mod_yearname_x` on train and test data separately (ensured transformer was only fit on the train set to prevent data leakage)
  - Ordinal and one hot encoded features on train and test data separately
  - Imputed and scaled `trav_sp` on train and test data separately (ensured transformer was only fit on the train set to prevent data leakage)
9. Removed variables that were not needed/useful: 
  - `makename` (too many levels), 
  - `wrk_zonename` (99% of one class)
  - `hour_imname` (unnecessary with `hour_binned`)
  - `vspd_lim` (unnecessary with `speeding_status`) 
  - `body_type_x` (unnecessary with `body_type_binned`)
  - `weathername` (unecessary with `weather_binned`)
  - `typ_intname` (unnecessary with `intersection_binned`)
  - `maxsev_imname` (unnecessary with `maxsev_binned`)
  - `trav_sp` (unnecessary with `trav_sp_imputed`)
10. Final dataset dimension 50535 rows, 119 columns (prior to train/test split)

## Read, clean the data

In [5]:
# Read in data from github
url_accident = "https://github.com/BradenAnderson/Accident_Severity_Prediction/blob/main/Data/accident.csv.gz?raw=tr"
url_vehicle = "https://github.com/BradenAnderson/Accident_Severity_Prediction/blob/main/Data/vehicle.csv.gz?raw=tr"
url_person = "https://github.com/BradenAnderson/Accident_Severity_Prediction/blob/main/Data/person.csv.gz?raw=tr"

accident = pd.read_csv(url_accident,compression='gzip')
vehicle = pd.read_csv(url_vehicle, compression='gzip', low_memory=False, encoding="ISO-8859-1")
person = pd.read_csv(url_person, compression='gzip', low_memory=False, encoding="ISO-8859-1")

In [6]:
# Filter accidents where driver is present and vehicle is involved
person = person.loc[(person.VEH_NO==1) & (person.PER_NO==1)]
vehicle = vehicle.loc[vehicle.VEH_NO==1]

In [7]:
# Left join person with vehicle and accident
# Duplicated CASENUM are dropped
df = person.merge(vehicle.drop_duplicates(subset=['CASENUM']), on='CASENUM', how='left')
df = df.merge(accident.drop_duplicates(subset=['CASENUM']),on='CASENUM',how='left')

In [8]:
# Comprehensive list of variables used in this analysis
# ORIGINAL features from lab 1 / EDA: regionname, urbanicityname, body_typname, makename, mod_yearname, vtrafwayname, vnum_lanname, vsurcondname, vtrafconname, 
                                # typ_intname, int_hwyname, weathername, wkdy_imname, reljct1_imname, lgtcon_imname, maxsev_imname, alchl_imname, age_im, sex_imname, trav_sp
# DERIVED features: hour_binned, speeding_status
# NEW features post-lab 1 / EDA: rest_usename, pcrash1_imname, weather_binned (binning of `weathername`), body_type_binned (binning of `body_typname`), int_binned (binning of `typ_intname`)
# DISCARDED features that were not useful: hour_imname (unnecessary with `hour_binned`), vspd_lim (unnecessary with `speeding_status`), makename (too many levels), wrk_zonename

df = df[['REGIONNAME','URBANICITYNAME','BODY_TYPNAME_x', 'MOD_YEARNAME_x','VTRAFWAYNAME','VNUM_LANNAME','VSURCONDNAME','VTRAFCONNAME','TYP_INTNAME','INT_HWYNAME','WEATHERNAME',
        'WKDY_IMNAME', 'RELJCT1_IMNAME','LGTCON_IMNAME','MAXSEV_IMNAME','ALCHL_IMNAME','AGE_IM','SEX_IMNAME','TRAV_SP','REST_USENAME','PCRASH1_IMNAME','HOUR_IMNAME','VSPD_LIM',
        'HOUR_IM']]

df = df.rename(columns=str.lower)
df.shape

(54473, 24)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54473 entries, 0 to 54472
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   regionname      54473 non-null  object 
 1   urbanicityname  54473 non-null  object 
 2   body_typname_x  54473 non-null  object 
 3   mod_yearname_x  54473 non-null  object 
 4   vtrafwayname    54427 non-null  object 
 5   vnum_lanname    54427 non-null  object 
 6   vsurcondname    54427 non-null  object 
 7   vtrafconname    54427 non-null  object 
 8   typ_intname     54473 non-null  object 
 9   int_hwyname     54473 non-null  object 
 10  weathername     54473 non-null  object 
 11  wkdy_imname     54473 non-null  object 
 12  reljct1_imname  54473 non-null  object 
 13  lgtcon_imname   54473 non-null  object 
 14  maxsev_imname   54473 non-null  object 
 15  alchl_imname    54473 non-null  object 
 16  age_im          54473 non-null  int64  
 17  sex_imname      54473 non-null 

In [10]:
df.head()

,regionname,urbanicityname,body_typname_x,mod_yearname_x,vtrafwayname,vnum_lanname,vsurcondname,vtrafconname,typ_intname,int_hwyname,...,maxsev_imname,alchl_imname,age_im,sex_imname,trav_sp,rest_usename,pcrash1_imname,hour_imname,vspd_lim,hour_im
0,"West (MT, ID, WA, OR, CA, NV, NM, AZ, UT, CO, ...",Rural Area,"4-door sedan, hardtop",2018,"Two-Way, Not Divided",Five lanes,Snow,Traffic control signal(on colors) not known wh...,Four-Way Intersection,No,...,No Apparent Injury (O),No Alcohol Involved,61,Female,25.0,Shoulder and Lap Belt Used,Going Straight,8:00am-8:59am,98.0,8
1,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",Urban Area,"4-door sedan, hardtop",2013,"Two-Way, Not Divided",Two lanes,Dry,No Controls,Not an Intersection,No,...,Suspected Minor Injury (B),No Alcohol Involved,23,Male,45.0,Shoulder and Lap Belt Used,Going Straight,1:00am-1:59am,25.0,1
2,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",Urban Area,Other or Unknown automobile type,Unknown,"Two-Way, Divided, Unprotected Median",Four lanes,Dry,No Controls,T-Intersection,No,...,No Apparent Injury (O),No Alcohol Involved,27,Female,15.0,Not Reported,Going Straight,1:00pm-1:59pm,45.0,13
3,"West (MT, ID, WA, OR, CA, NV, NM, AZ, UT, CO, ...",Rural Area,"Compact Utility (Utility Vehicle Categories ""S...",2015,"Two-Way, Divided, Positive Median Barrier",Two lanes,Snow,No Controls,Not an Intersection,Yes,...,No Apparent Injury (O),No Alcohol Involved,20,Male,65.0,Shoulder and Lap Belt Used,Going Straight,2:00pm-2:59pm,80.0,14
4,"Northeast (PA, NJ, NY, NH, VT, RI, MA, ME, CT)",Rural Area,Station Wagon (excluding van and truck based),2004,Not Reported,Not Reported,Snow,Warning Sign,Not an Intersection,No,...,No Apparent Injury (O),No Alcohol Involved,23,Male,998.0,Shoulder and Lap Belt Used,Negotiating a Curve,5:00pm-5:59pm,50.0,17


In [11]:
# Check for NA values
df.isnull().sum()

regionname         0
urbanicityname     0
body_typname_x     0
mod_yearname_x     0
vtrafwayname      46
vnum_lanname      46
vsurcondname      46
vtrafconname      46
typ_intname        0
int_hwyname        0
weathername        0
wkdy_imname        0
reljct1_imname     0
lgtcon_imname      0
maxsev_imname      0
alchl_imname       0
age_im             0
sex_imname         0
trav_sp           46
rest_usename       0
pcrash1_imname    46
hour_imname        0
vspd_lim          46
hour_im            0
dtype: int64

In [12]:
# Remove NA values since they are low representation of the entire dataset
df.dropna(inplace=True)

In [13]:
# Confirm NA values are removed
df.isnull().sum().sum()

0

In [14]:
df.shape

(54427, 24)

### Inspect each feature

Data dimension is 54427x24 prior to inspection.

#### Year of vehicle **(Continuous)**
Data cleaning: remove `unknown` instances, filter years to 1980+

In [15]:
# mod_yearname_x
df.mod_yearname_x.value_counts()

2016       3809
2017       3722
2019       3489
2018       3356
Unknown    2935
           ... 
1972          1
1932          1
1953          1
1959          1
1969          1
Name: mod_yearname_x, Length: 67, dtype: int64

In [16]:
# Remove unknown instances, convert to numeric dtype
non_int_values = pd.to_numeric(df.mod_yearname_x, errors='coerce').isna()
df = df[-non_int_values]
df['mod_yearname_x'] = pd.to_numeric(df['mod_yearname_x'])

In [17]:
# Filter to years 1980+
df = df.loc[df['mod_yearname_x'] >= 1980]

In [18]:
df.shape

(50930, 24)

#### Age of driver **(Continuous)**
Data cleaning: remove observations below 15 years old.

In [19]:
# age_im
df = df.loc[df['age_im'] > 15]

In [20]:
df.shape

(50705, 24)

#### Traveling speed of vehicle **(Continuous)**

Data cleaning: none  
28975 observations will be lost if `speed greater than 151 mph`, `speed unreported`, `speed reported as unknown` were removed. This is a huge chunk of data so we will impute during preprocessing step.

In [21]:
# trav_sp
# Speed Greater Than 151 mph indicated as 997
# Speed not reported indicated as 998
# Speed reported as unknown indicated as 999
# Discern number of rows lost if 997, 998, and 998 are removed
# 28975 !! Will impute instead
df.loc[df['trav_sp'] >= 997].shape[0]

28975

#### Sex of driver

Data cleaning: none

In [22]:
# sex_imname
df.sex_imname.value_counts()

Male      31519
Female    19186
Name: sex_imname, dtype: int64

#### Region where crash occurred

Data cleaning: keep regional name, remove respective states

- Northeast (PA, NJ, NY, NH, VT, RI, MA, ME, CT)
- West (MT, ID, WA, OR, CA, NV, NM, AZ, UT, CO, WY, AK, HI)
- Midwest (OH, IN, IL, MI, WI, MN, ND, SD, NE, IA, MO, KS)
- South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA, FL, AL, MS, LA, AR, OK, TX)


In [23]:
# regionname
df.loc[:, "regionname"] = df.loc[:,"regionname"].apply(lambda string: string.split()[0])
df.regionname.value_counts()

South        26481
Midwest       9643
West          8449
Northeast     6132
Name: regionname, dtype: int64

#### Geographical area of the crash

Data cleaning: keep urban/rural indication, remove "Area" that proceeded it

- Urban
- Rural

In [24]:
# urbanicityname
df["urbanicityname"] = df.loc[:,"urbanicityname"].apply(lambda string: string.split()[0])
df.urbanicityname.value_counts()

Urban    37472
Rural    13233
Name: urbanicityname, dtype: int64

#### Trafficway flow just prior to crash
Data cleaning: bin `Not Reported`, `Reported as Unknown` into `NR/UNK`

In [25]:
# vtrafwayname
df.vtrafwayname.value_counts()

Two-Way, Not Divided                                     21560
Not Reported                                              8451
Two-Way,  Divided, Positive  Median Barrier               8346
Two-Way, Divided, Unprotected Median                      6455
Two-Way, Not Divided With a Continuous Left-Turn Lane     1860
Non-Trafficway or Driveway Access                         1801
Entrance/Exit Ramp                                        1217
One-Way Trafficway                                         997
Reported as Unknown                                         18
Name: vtrafwayname, dtype: int64

In [26]:
# Bin observations 'Not Reported', 'Reported as Unknown' into NR/UNK
df['vtrafwayname'] = df['vtrafwayname'].replace(['Not Reported', 'Reported as Unknown'], 'NR/UNK')

In [27]:
df.vtrafwayname.value_counts()

Two-Way, Not Divided                                     21560
NR/UNK                                                    8469
Two-Way,  Divided, Positive  Median Barrier               8346
Two-Way, Divided, Unprotected Median                      6455
Two-Way, Not Divided With a Continuous Left-Turn Lane     1860
Non-Trafficway or Driveway Access                         1801
Entrance/Exit Ramp                                        1217
One-Way Trafficway                                         997
Name: vtrafwayname, dtype: int64

#### Number of travel lanes just prior to crash. 

Data cleaning: bin `Not Reported`, `Reported as Unknown` as NR/UNK
- Median: lanes in opposite directions are additive. 
- No median: lanes in traveling direction counts

In [28]:
# vnum_lanname
# Bin 'Not Reported', 'Reported as Unknown' as NR/UNK
df.vnum_lanname.value_counts()

Two lanes                            18000
Not Reported                         14340
Three lanes                           5775
Four lanes                            4714
Five lanes                            3535
Non-Trafficway or Driveway Access     1801
Six lanes                             1200
One lane                               917
Seven or more lanes                    407
Reported as Unknown                     16
Name: vnum_lanname, dtype: int64

In [29]:
# Bin 'Not Reported', 'Reported as Unknown' as NR/UNK
df['vnum_lanname'] = df['vnum_lanname'].replace(['Not Reported', 'Reported as Unknown'], 'NR/UNK')

In [30]:
df.vnum_lanname.value_counts()

Two lanes                            18000
NR/UNK                               14356
Three lanes                           5775
Four lanes                            4714
Five lanes                            3535
Non-Trafficway or Driveway Access     1801
Six lanes                             1200
One lane                               917
Seven or more lanes                    407
Name: vnum_lanname, dtype: int64

#### Roadway surface condition just prior to crash

Data cleaning: bin `Not Reported`, `Reported as Unknown`, `Other` as NR/UNK/OTH

In [31]:
# vsurcondname
df.vsurcondname.value_counts()

Dry                                  37880
Wet                                   6681
Not Reported                          3165
Non-Trafficway or Driveway Access     1801
Snow                                   487
Ice/Frost                              325
Water (Standing or Moving)             102
Slush                                   92
Mud, Dirt or Gravel                     87
Reported as Unknown                     59
Sand                                    15
Other                                    7
Oil                                      4
Name: vsurcondname, dtype: int64

In [32]:
# Bin 'Not Reported', 'Reported as Unknown', 'Other' as NR/UNK/OTH
df['vsurcondname'] = df['vsurcondname'].replace(['Not Reported', 'Reported as Unknown', 'Other'], 'NR/UNK/OTH')

In [33]:
df.vsurcondname.value_counts()

Dry                                  37880
Wet                                   6681
NR/UNK/OTH                            3231
Non-Trafficway or Driveway Access     1801
Snow                                   487
Ice/Frost                              325
Water (Standing or Moving)             102
Slush                                   92
Mud, Dirt or Gravel                     87
Sand                                    15
Oil                                      4
Name: vsurcondname, dtype: int64

#### Traffic controls in the vehicle’s environment just prior to crash
Data cleaning: bin `Not Reported`, `Other Regulatory Sign`, `Other`, `Reported as Unknown`, `Other Highway Traffic Signal`, `Unknown Highway Traffic Signal`, `Unknown Regulatory Sign` as `NR/UNK/OTH`

In [34]:
# vtrafconname
df.vtrafconname.value_counts()

No Controls                                                                     25691
Traffic control signal(on colors) not known whether or not Pedestrian Signal    10520
Not Reported                                                                     7737
Stop Sign                                                                        4768
Yield Sign                                                                        580
Traffic control signal (on colors) with Pedestrian Signal                         520
Warning Sign                                                                      251
Traffic control signal (on colors) without Pedestrian Signal                      183
Flashing Traffic Control Signal                                                   128
Other Regulatory Sign                                                              94
Other                                                                              67
Railway Crossing Device                               

In [35]:
# Bin 'Not Reported', 'Other Regulatory Sign', 'Other', 'Reported as Unknown', 'Other Highway Traffic Signal', 'Unknown Highway Traffic Signal', 'Unknown Regulatory Sign'
df['vtrafconname'] = df['vtrafconname'].replace(['Not Reported', 'Other Regulatory Sign', 'Other', 
                                                 'Reported as Unknown', 'Other Highway Traffic Signal', 
                                                 'Unknown Highway Traffic Signal', 'Unknown Regulatory Sign'], 'NR/UNK/OTH')

In [36]:
df.vtrafconname.value_counts()

No Controls                                                                     25691
Traffic control signal(on colors) not known whether or not Pedestrian Signal    10520
NR/UNK/OTH                                                                       7943
Stop Sign                                                                        4768
Yield Sign                                                                        580
Traffic control signal (on colors) with Pedestrian Signal                         520
Warning Sign                                                                      251
Traffic control signal (on colors) without Pedestrian Signal                      183
Flashing Traffic Control Signal                                                   128
Railway Crossing Device                                                            49
Person                                                                             47
School Zone Sign/Device                               

#### Did crash occur at intersection
Data cleaning: drop `Unknown`

In [37]:
# int_hwyname
# Remove 'Unknown'
df.int_hwyname.value_counts()

No         46035
Yes         4663
Unknown        7
Name: int_hwyname, dtype: int64

In [38]:
# Drop observations with Unknown
df.drop(df[df['int_hwyname'] == 'Unknown'].index, inplace = True)
df.int_hwyname.value_counts()

No     46035
Yes     4663
Name: int_hwyname, dtype: int64

In [39]:
df.shape

(50698, 24)

#### Name of weekday where crash occurred
Data cleaning: none

In [40]:
# wkdy_imname
df.wkdy_imname.value_counts()

Friday       8427
Thursday     7682
Wednesday    7631
Tuesday      7239
Saturday     6986
Monday       6938
Sunday       5795
Name: wkdy_imname, dtype: int64

#### Relation to junction (crash's location with respect to presence in an interchange area)
Data cleaning: none

In [41]:
# reljct1_imname
df.reljct1_imname.value_counts()

No     47293
Yes     3405
Name: reljct1_imname, dtype: int64

#### Lighting condition during time of crash
Data cleaning: none

In [42]:
# lgtcon_imname
df.lgtcon_imname.value_counts()

Daylight                   34052
Dark - Lighted              8468
Dark - Not Lighted          5762
Dusk                        1292
Dawn                         774
Dark - Unknown Lighting      339
Other                         11
Name: lgtcon_imname, dtype: int64

#### Alcohol state of driver
Data cleaning: none

In [43]:
# alchl_imname
df.alchl_imname.value_counts()

No Alcohol Involved    46713
Alcohol Involved        3985
Name: alchl_imname, dtype: int64

### Newly derived features or careful binning of existing features

#### Speeding status of driver (Derived feature)
Ascertained whether the driver was speeding or not from `trav_sp` (traveling speed of car) and `vspd_lim` (posted speed limit) as new feature `speeding_status`

In [44]:
def create_speeding_feature(row):
    
    # Speed greater than 151 mph, you're speeding
    if row["trav_sp"] == 997:
        return 'speeding'
    
    # If we know they were going 95 mph or faster, calling that speeding
    # regardless of what the speed limit is, or if we even know ths speed limit.
    elif row["trav_sp"] < 152 and row["trav_sp"] >= 95:
        return 'speeding'
    
    # If the speed limit is unknown, speeding is unknown
    elif row["vspd_lim"] == 98 or row["vspd_lim"] == 99:
        return 'unknown'
    
    # If the traveling speed is unknown, speeding is unknown
    elif row['trav_sp'] == 998 or row['trav_sp'] == 999:
        return 'unknown'
    
    # If traveling faster than the speed limit, speeding
    elif row['trav_sp'] > row['vspd_lim']:
        return 'speeding'
    else:
        return 'not speeding'
    
df["speeding_status"] = df.apply(lambda row: create_speeding_feature(row), axis='columns')

In [45]:
df.speeding_status.value_counts()

unknown         30304
not speeding    17151
speeding         3243
Name: speeding_status, dtype: int64

#### Time of accident, hours binned (Derived feature)
Condensed `hour_im` (hour of accident based on 24 hours) into 4 categories:

- Morning (6am-noon)
- Afternoon (noon-6pm)
- Evening (6pm-midnight)
- Night (midnight-6am)

In [46]:
def create_binned_hours_feature(numeric_hour, night_hours=[0,1,2,3,4,21,22,23], morning_hours=[5,6,7,8,9,10,11], 
                                afternoon_hours=[12,13,14,15], evening_hours=[16,17,18,19,20]):
    
    # NOTE: numeric_hour is a value from the HOUR_IM column

    # Default for night is 9pm-4:59am
    if numeric_hour in night_hours:
        return "Night"
    
    # Default for morning is 5am-11:59am
    elif numeric_hour in morning_hours:
        return "Morning"
    
    # Default for Afternoon is 12:00pm-3:59pm
    elif numeric_hour in afternoon_hours:
        return "Afternoon"
    
    # Default for evening is 4:00pm-8:59pm
    elif numeric_hour in evening_hours:
        return "Evening"
    
df["hour_binned"] = df["hour_im"].apply(lambda hour: create_binned_hours_feature(numeric_hour=hour))

# After hour_im is used to create hour_binned, we no longer need hour_im
df.drop(columns="hour_im", inplace=True)

In [47]:
df.hour_binned.value_counts()

Evening      16618
Afternoon    13563
Morning      12804
Night         7713
Name: hour_binned, dtype: int64

#### Body type of vehicle (Careful binning)
Condensed 62 categories of vehicles into 12


In [48]:
df.body_typname_x.value_counts()

4-door sedan, hardtop                                                                              17915
Compact Utility (Utility Vehicle Categories "Small" and "Midsize")                                  7677
Light Pickup                                                                                        6506
Station Wagon (excluding van and truck based)                                                       2307
Large utility (ANSI D16.1 Utility Vehicle Categories and "Full Size" and "Large")                   2178
                                                                                                   ...  
Unknown if single-unit or combination unit Medium Truck (GVWR range 10,001 lbs. to 26,000 lbs.)        1
Unknown if single-unit or combination unit Heavy Truck (GVWR greater than 26,000 lbs.)                 1
Large Limousine-more than four side doors or stretched chassis                                         1
Light Vehicle Based Motor Home (chassis mounted)       

In [49]:
df['body_type_binned']=df['body_typname_x']

df['body_type_binned']=df['body_type_binned'].replace(to_replace=
                                        ['4-door sedan, hardtop',
                                       '2-door sedan,hardtop,coupe',
                                       '3-door coupe','Sedan/Hardtop, number of doors unknown'
                                        ],value=1)
df['body_type_binned']=df['body_type_binned'].replace(to_replace=
                                        'Compact Utility (Utility Vehicle Categories \"Small" and \"Midsize\")',
                                         value=2)
df['body_type_binned']=df['body_type_binned'].replace(to_replace=
                                                      ["Auto-based pickup (includes E1 Camino, Caballero, Ranchero, SSR, G8-ST, Subaru Brat, Rabbit Pickup)",
                                        "Light Pickup",
                                        "Unknown (pickup style) light conventional truck type",
                                        "Unknown light truck type",
                                        "Unknown light vehicle type (automobile,utility vehicle, van, or light truck)",
                                        'Light Vehicle Based Motor Home (chassis mounted)'
                                        ],value=3)
df['body_type_binned']=df['body_type_binned'].replace(to_replace=
                                         ['Large utility (ANSI D16.1 Utility Vehicle Categories and "Full Size" and "Large")',
                                        'Utility Vehicle, Unknown body type'
                                         ],value=4)
df['body_type_binned']=df['body_type_binned'].replace(to_replace=
                                        ['ATV/ATC [All-Terrain Cycle]',
                                        'Moped or motorized bicycle',
                                        'Motor Scooter',
                                        'Off-road Motorcycle',
                                        'Other motored cycle type (mini-bikes, pocket motorcycles "pocket bikes")',
                                        'Three-wheel Motorcycle (2 Rear Wheels)',
                                        'Two Wheel Motorcycle (excluding motor scooters)',
                                        'Unenclosed Three Wheel Motorcycle / Unenclosed Autocycle (1 Rear Wheel)',
                                        'Unknown motored cycle type',
                                        'Unknown Three Wheel Motorcycle Type'
                                        ],value=5)
df['body_type_binned']=df['body_type_binned'].replace(to_replace=
                                        ['Station Wagon (excluding van and truck based)',
                                        'Utility station wagon (includes suburban limousines, Suburban, Travellall, Grand Wagoneer)'
                                        ],value=6)
df['body_type_binned']=df['body_type_binned'].replace(to_replace=
                                        ['3-door/2-door hatchback',
                                        '5-door/4-door hatchback',
                                        'Hatchback, number of doors unknown'
                                        ],value=7)
df['body_type_binned']=df['body_type_binned'].replace(to_replace=
                                        ['Cross Country/Intercity Bus',
                                        'Medium/Heavy Vehicle Based Motor Home',
                                        'Other Bus Type',
                                        'School Bus',
                                        'Transit Bus (City Bus)',
                                        'Unknown Bus Type'
                                        ],value=8)
df['body_type_binned']=df['body_type_binned'].replace(to_replace=
                                        ['Cab Chassis Based (includes Rescue Vehicle, Light Stake, Dump, and Tow Truck)',
                                        'Medium/heavy Pickup (GVWR greater than 10,000 lbs.)',
                                        'Single-unit straight truck or Cab-Chassis (GVWR greater than 26,000 lbs.)',
                                        'Single-unit straight truck or Cab-Chassis (GVWR range 10,001 to 19,500 lbs.)',
                                        'Single-unit straight truck or Cab-Chassis (GVWR range 19,501 to 26,000 lbs.)',
                                        'Single-unit straight truck or Cab-Chassis (GVWR unknown)',
                                        'Truck-tractor (Cab only, or with any number of trailing unit; any weight)',
                                        'Unknown if single-unit or combination unit Heavy Truck (GVWR greater than 26,000 lbs.)',
                                        'Unknown if single-unit or combination unit Medium Truck (GVWR range 10,001 lbs. to 26,000 lbs.)',
                                        'Unknown medium/heavy truck type'
                                         ],value=9)
df['body_type_binned']=df['body_type_binned'].replace(to_replace=
                                        'Convertible(excludes sun-roof,t-bar)',
                                           value=10)
df['body_type_binned']=df['body_type_binned'].replace(to_replace=
                                        ['Large Van-Includes van-based buses (B150-B350, Sportsman, Royal Maxiwagon, Ram, Tradesman,...)',
                                        'Minivan (Chrysler Town and Country, Caravan, Grand Caravan, Voyager, Voyager, Honda-Odyssey, ...)',
                                        'Other van type (Hi-Cube Van, Kary)',
                                        'Step van (GVWR greater than 10,000 lbs.)',
                                        'Step-van or walk-in van (GVWR less than or equal to 10,000 lbs.)',
                                        'Unknown van type',
                                        'Van-Based Bus GVWR greater than 10,000 lbs.'
                                        ],value=11)
df['body_type_binned']=df['body_type_binned'].replace(to_replace=
                                        ['Construction equipment other than trucks (includes graders)',
                                        'Farm equipment other than trucks',
                                        'Golf Cart',
                                        'Large Limousine-more than four side doors or stretched chassis',
                                        'Low Speed Vehicle (LSV) / Neighborhood Electric Vehicle (NEV)',
                                        'Not Reported',
                                        'Other or Unknown automobile type',
                                        'Other vehicle type (includes go-cart, fork-lift, city street sweeper dunes/swamp buggy)',
                                        'Recreational Off-Highway Vehicle',
                                        'Unknown body type',
                                        'Unknown truck type (light/medium/heavy)',
                                        ],value=12)


df['body_type_binned'] = df['body_type_binned'].astype(str)

##drop original column after binning
df.drop(columns='body_typname_x', inplace=True)

In [50]:
df.body_type_binned.value_counts()

1     19921
3      8074
2      7677
5      2615
4      2450
6      2429
9      2075
11     2055
7      1625
12     1273
10      355
8       149
Name: body_type_binned, dtype: int64

#### Type of intersection (Careful binning)
Condensed 11 types of intersection into two (whether accident took place at an intersection).

In [51]:
df.typ_intname.value_counts()

Not an Intersection        28229
Four-Way Intersection      12518
T-Intersection              5235
Not Reported                4201
Y-Intersection               161
Five Point, or More          152
Roundabout                   139
Traffic Circle                34
L-Intersection                21
Other Intersection Type        6
Reported as Unknown            2
Name: typ_intname, dtype: int64

In [52]:
def intersection_category(row):
    if row == 'Not an Intersection':
        result = 'No'
    elif row == 'Reported as Unknown':
        result = 'Other'
    elif row == 'Not Reported':
        result = 'Other'
    else:
        result = 'Yes'
    return result

df['intersection_binned'] = df['typ_intname'].apply(intersection_category)
df.intersection_binned.value_counts()

No       28229
Yes      18266
Other     4203
Name: intersection_binned, dtype: int64

#### Weather during time of crash (Careful binning)
Condensed 13 types of weather/atmospheric situation into six:

- Clear        
- Not Clear     
- Rain          
- Other         
- Wintery        
- Windy

In [53]:
# weathername
df.weathername.value_counts()

Clear                       35486
Cloudy                       6769
Rain                         4689
Not Reported                 2569
Snow                          752
Fog, Smog, Smoke              216
Reported as Unknown            70
Severe Crosswinds              46
Blowing Snow                   33
Sleet or Hail                  29
Freezing Rain or Drizzle       19
Other                          17
Blowing Sand, Soil, Dirt        3
Name: weathername, dtype: int64

In [54]:
def weather_cat(row):
    if row == 'Cloudy':
        return 'Not Clear'
    elif row == 'Fog, Smog, Smoke':
        return 'Not Clear' 
    elif row == 'Snow':
        return 'Wintery'
    elif row == 'Blowing Snow':
        return 'Wintery'
    elif row == 'Sleet or Hail':
        return 'Wintery'
    elif row == 'Freezing Rain or Drizzle':
        return 'Wintery'
    elif row == 'Severe Crosswinds':
        return 'Windy'
    elif row == 'Blowing Sand, Soil, Dirt':
        return 'Windy'
    elif row == 'Clear':
        return 'Clear'
    elif row == 'Rain':
        return 'Rain'
    else:
        return 'Other'
df['weather_binned'] = df['weathername'].apply(weather_cat)
df.weather_binned.value_counts()

Clear        35486
Not Clear     6985
Rain          4689
Other         2656
Wintery        833
Windy           49
Name: weather_binned, dtype: int64

#### Restraint use by driver (Newly added feature)
Added new feature and condensed eight types of restraint use into four:

- Minimal
- Full
- None
- Other

In [55]:
# rest_use
df.rest_usename.value_counts()

Shoulder and Lap Belt Used       40587
None Used/Not Applicable          4735
Reported as Unknown               3221
Not Reported                      1297
Lap Belt Only Used                 418
Shoulder Belt Only Used            331
Restraint Used - Type Unknown      104
Other                                5
Name: rest_usename, dtype: int64

In [56]:
# rest_use: bin into none, minimal, full, other
def restraint_category(row):
    if row == 'Shoulder and Lap Belt Used':
        result = 'Full'
    elif row == 'None Used/Not Applicable':
        result = 'None'    
    elif row == 'Lap Belt Only Used':
        result = 'Minimal'
    elif row == 'Shoulder Belt Only Used':
        result = 'Minimal'
    elif row == 'Restraint Used - Type Unknown':
        result = 'Minimal'
    else:
        result = 'Other'
    return result

df['restraint_binned'] = df['rest_usename'].apply(restraint_category)
df.restraint_binned.value_counts()

Full       40587
None        4735
Other       4523
Minimal      853
Name: restraint_binned, dtype: int64

#### What driver was doing right before crash (Newly added feature)

In [57]:
# pcrash1_imname
df.pcrash1_imname.value_counts()

Going Straight                                                28172
Turning Left                                                   7056
Negotiating a Curve                                            4199
Changing Lanes                                                 2779
Turning Right                                                  2593
Stopped in Roadway                                             1268
Decelerating in Road                                           1102
Backing Up (other than for Parking Position)                    926
Passing or Overtaking Another Vehicle                           726
Starting in Road                                                639
Making a U-turn                                                 454
Merging                                                         265
Leaving a Parking Position                                      169
Entering a Parking Position                                     112
Accelerating in Road                            

#### **Response variable:** Maximum injury severity of driver (Careful binning)

1. Removed `Injured, Severity Unknown` and `Died Prior to Crash*`
2. Condensed remaining five categories into two: `Fatal` and `Not Fatal`
3. Class imbalance between fatal (15%) and not fatal (85%). We consider fatal as the "success".
4. Visualize plot below

In [58]:
# maxsev_imname
df.maxsev_imname.value_counts()

No Apparent Injury (O)          22490
Possible Injury (C)             11649
Suspected Minor Injury (B)       9012
Suspected Serious Injury (A)     6037
Fatal Injury (K)                 1347
Injured, Severity Unknown         162
Died Prior to Crash*                1
Name: maxsev_imname, dtype: int64

In [59]:
# Remove Injured, Severity Unknown and Died Prior to Crash 
remove = ['Injured, Severity Unknown', 'Died Prior to Crash*']
df = df[df.maxsev_imname.isin(remove) == False]
df['maxsev_imname'] = df['maxsev_imname'].str[:-4]
df.maxsev_imname.value_counts()

No Apparent Injury          22490
Possible Injury             11649
Suspected Minor Injury       9012
Suspected Serious Injury     6037
Fatal Injury                 1347
Name: maxsev_imname, dtype: int64

In [60]:
# Combine No Apparent Injury, Possible Injury, Suspected Minor Injury into one bin
# Combine Suspected Serious Injury and Fatal Injury into one bin
df['maxsev_binned'] = df['maxsev_imname'].replace(to_replace={'Suspected Serious Injury':'Fatal', 
                                                              'Fatal Injury':'Fatal', 
                                                              'No Apparent Injury':'Not Fatal', 
                                                              'Possible Injury':'Not Fatal', 
                                                              'Suspected Minor Injury':'Not Fatal'})
df.maxsev_binned.value_counts()

Not Fatal    43151
Fatal         7384
Name: maxsev_binned, dtype: int64

### End of data cleaning. Review final dataset

In [62]:
# Remove features that are not useful for modeling or redundant post binning
# Recall from summary those features are:
#  - `makename` (too many levels)
#  - `wrk_zonename` (99% of one class)
#  - `hour_imname` (unnecessary with `hour_binned`)
#  - `vspd_lim` (unnecessary with `speeding_status`) 
#  - `body_type_x` (unnecessary with `body_type_binned`)
#  - `weathername` (unecessary with `weather_binned`)
#  - `typ_intname` (unnecessary with `intersection_binned`)
#  - `maxsev_imname` (unnecessary with `maxsev_binned`)
#  - `trav_sp` (unnecessary with `trav_sp_imputed`) this will be removed later after imputation

df = df[df.columns.difference(['body_typname_x', 'typ_intname', 'weathername', 'rest_usename', 'hour_imname', 'vspd_lim'])]

In [63]:
# Check for duplicates
# Recall duplicated casenum was taken cared of during merge
# Here we check for duplicates of instances excluding `casenum`
df.duplicated().sum()

0

In [64]:
# Confirm there are zero NA values
df.isnull().sum().sum()

0

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50535 entries, 0 to 54471
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age_im               50535 non-null  int64  
 1   alchl_imname         50535 non-null  object 
 2   body_type_binned     50535 non-null  object 
 3   hour_binned          50535 non-null  object 
 4   int_hwyname          50535 non-null  object 
 5   intersection_binned  50535 non-null  object 
 6   lgtcon_imname        50535 non-null  object 
 7   maxsev_binned        50535 non-null  object 
 8   maxsev_imname        50535 non-null  object 
 9   mod_yearname_x       50535 non-null  int64  
 10  pcrash1_imname       50535 non-null  object 
 11  regionname           50535 non-null  object 
 12  reljct1_imname       50535 non-null  object 
 13  restraint_binned     50535 non-null  object 
 14  sex_imname           50535 non-null  object 
 15  speeding_status      50535 non-null 

In [66]:
df.head()

,age_im,alchl_imname,body_type_binned,hour_binned,int_hwyname,intersection_binned,lgtcon_imname,maxsev_binned,maxsev_imname,mod_yearname_x,...,sex_imname,speeding_status,trav_sp,urbanicityname,vnum_lanname,vsurcondname,vtrafconname,vtrafwayname,weather_binned,wkdy_imname
0,61,No Alcohol Involved,1,Morning,No,Yes,Daylight,Not Fatal,No Apparent Injury,2018,...,Female,unknown,25.0,Rural,Five lanes,Snow,Traffic control signal(on colors) not known wh...,"Two-Way, Not Divided",Not Clear,Wednesday
1,23,No Alcohol Involved,1,Night,No,No,Dark - Not Lighted,Not Fatal,Suspected Minor Injury,2013,...,Male,speeding,45.0,Urban,Two lanes,Dry,No Controls,"Two-Way, Not Divided",Clear,Wednesday
3,20,No Alcohol Involved,2,Afternoon,Yes,No,Daylight,Not Fatal,No Apparent Injury,2015,...,Male,not speeding,65.0,Rural,Two lanes,Snow,No Controls,"Two-Way, Divided, Positive Median Barrier",Wintery,Wednesday
4,23,No Alcohol Involved,6,Evening,No,No,Dark - Not Lighted,Not Fatal,No Apparent Injury,2004,...,Male,unknown,998.0,Rural,NR/UNK,Snow,Warning Sign,NR/UNK,Wintery,Wednesday
5,57,No Alcohol Involved,3,Evening,No,No,Dark - Not Lighted,Not Fatal,Possible Injury,2003,...,Female,unknown,998.0,Rural,NR/UNK,Dry,No Controls,NR/UNK,Clear,Wednesday


In [67]:
# Summary statistics of continuous variables
# Ignore trav_sp because of indicator values e.g., 999 represents speed reported as unknown
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age_im,50535.0,39.552548,17.556665,16.0,25.0,35.0,52.0,120.0
mod_yearname_x,50535.0,2010.730167,6.848490,1980.0,2006.0,2012.0,2017.0,2021.0
trav_sp,50535.0,583.428930,479.172085,0.0,35.0,998.0,998.0,999.0


In [68]:
# Summary statistics of categorical variables
# This gives reader a cursory view of the average situation/environment of a typical crash
df.describe(include=[object], exclude=[np.number]).T

,count,unique,top,freq
alchl_imname,50535,2,No Alcohol Involved,46578
body_type_binned,50535,12,1,19865
hour_binned,50535,4,Evening,16566
int_hwyname,50535,2,No,45885
intersection_binned,50535,3,No,28126
lgtcon_imname,50535,7,Daylight,33942
maxsev_binned,50535,2,Not Fatal,43151
maxsev_imname,50535,5,No Apparent Injury,22490
pcrash1_imname,50535,19,Going Straight,28071
regionname,50535,4,South,26454


## Begin data preprocessing

In [69]:
# Split data 80/20 train test with 10 fold stratified cross validation
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["maxsev_binned"])

In [70]:
# instantiate the scaler
scaler = StandardScaler()

# Fit the scaler on the training set age and model year columns, and transform (scale) the training set age and model year columns
train_df.loc[:, ["age_im", "mod_yearname_x"]] = scaler.fit_transform(X=train_df.loc[:, ["age_im", "mod_yearname_x"]])

# Using means and standard deviations from training set, scale the test set
test_df.loc[:, ["age_im", "mod_yearname_x"]] = scaler.transform(X=test_df.loc[:, ["age_im", "mod_yearname_x"]])

In [71]:
# List of features to OneHotEncode
features_to_ohe = ["hour_binned", "intersection_binned", "lgtcon_imname", "pcrash1_imname", "regionname", 
                   "restraint_binned", "vtrafconname", "vtrafwayname", "weather_binned", "wkdy_imname", 
                   "vnum_lanname", "vsurcondname", "speeding_status", "body_type_binned"]

# Features that need to become binary
binary_features = ["urbanicityname", "alchl_imname", "reljct1_imname", "int_hwyname", "sex_imname"]

# List of all the features that are not being converted to binary, or being OneHotEncoder
# (Just a list of all features not in either of the two lists above).
non_encoded_features = [col for col in df.columns if col not in features_to_ohe + binary_features]

# Map each soon to be binary feature to its two initial values
binary_encoding_map = {feature:df[feature].unique().tolist() for feature in binary_features}

# Instantiate the OrdinalEncoder that will create the binary columns
ord_encoder = OrdinalEncoder(categories=[value for value in binary_encoding_map.values()])

# Map each feature that will be OneHotEncoded to a list of all possible categories that feature can take on. 
oridnal_encoding_map = {feature:df[feature].unique().tolist() for feature in features_to_ohe}

# Instantiate the OneHotEncoder, telling it what the possible categories are for each feature.
oh_encoder = OneHotEncoder(categories=[value for value in oridnal_encoding_map.values()]) 

encoders = ColumnTransformer(transformers=[("ordinal_encoder", ord_encoder, binary_features), 
                                           ("one_hot_encoder", oh_encoder, features_to_ohe)], 
                             remainder="passthrough",
                             sparse_threshold=0,
                             n_jobs=-1)

# Create a dataframe of all the encoded columns in the training set
train_encoded_features = pd.DataFrame(encoders.fit_transform(train_df.drop(columns=non_encoded_features)), 
                                      columns=encoders.get_feature_names_out())

# Remove the non-encoded versions, and add in the encoded-versions of all columns.
# Creates a version of the training set where all columns that need to be encoded are now encoded.
train_encoded_df = pd.concat(objs=[train_df.drop(columns=features_to_ohe+binary_features).reset_index(drop=True), 
                                   train_encoded_features.reset_index(drop=True)], 
                             axis='columns')

# Create a dataframe of all encoded (one hot and ordinal) columns for the test set
test_encoded_features = pd.DataFrame(encoders.transform(test_df.drop(columns=non_encoded_features)), 
                                     columns=encoders.get_feature_names_out())

# Remove the non-encoded versions, and add in the encoded-versions of all columns.
# Creates a version of the test set where all columns that need to be encoded are now encoded.
test_encoded_df = pd.concat(objs=[test_df.drop(columns=features_to_ohe+binary_features).reset_index(drop=True), 
                                   test_encoded_features.reset_index(drop=True)], 
                             axis='columns')

AttributeError: Transformer ordinal_encoder (type OrdinalEncoder) does not provide get_feature_names_out.

#### Imputing travel speed

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Set the placeholder values (which need to be imputed) to NaN
train_encoded_df.loc[train_encoded_df["trav_sp"].isin([997, 998, 999]), "trav_sp"] = np.nan
test_encoded_df.loc[test_encoded_df["trav_sp"].isin([997, 998, 999]), "trav_sp"] = np.nan

# Set up version of training dataset to be used for imputing trav_sp
imputer_train_df = train_encoded_df.copy(deep=True)
imputer_train_df.drop(columns=["maxsev_binned"], inplace=True)
severity_ord_enc = OrdinalEncoder(categories=[["No Apparent Injury", "Possible Injury", "Suspected Minor Injury", "Suspected Serious Injury", "Fatal Injury"]])
imputer_train_df["maxsev_imname"] = severity_ord_enc.fit_transform(X=imputer_train_df["maxsev_imname"].to_numpy().reshape(-1,1))

# Setting the minimum and maximum value the imputer can impute for a traveling speed. 
min_trav_sp = 15
max_trav_sp = 151
impute_min_values = [-np.inf if col != "trav_sp" else min_trav_sp for col in imputer_train_df.columns]
impute_max_values = [np.inf if col != "trav_sp" else max_trav_sp for col in imputer_train_df.columns]

# Instantiate the iterative imputer
imputer = IterativeImputer(min_value=impute_min_values, 
                           max_value=impute_max_values,
                           random_state=42)

# Fit the iterative imputer on the training set, and impute the training set
imputer_train_result = imputer.fit_transform(X=imputer_train_df)

# Take the output of running iteratrive imputer on the train set, and convert it to a dataframe
# Save the imputed version of traveling speed to train_encoded_df
imputer_train_result_df = pd.DataFrame(imputer_train_result, 
                                       columns=imputer.get_feature_names_out())
train_encoded_df["trav_sp_imputed"] = imputer_train_result_df["trav_sp"]

# Set up the version of the test dataset that will run through the imputer
imputer_test_df = test_encoded_df.copy(deep=True)
imputer_test_df.drop(columns=["maxsev_binned"], inplace=True)
imputer_test_df["maxsev_imname"] = severity_ord_enc.transform(X=imputer_test_df["maxsev_imname"].to_numpy().reshape(-1,1))

# Run the test data through the imputer
imputer_test_result = imputer.transform(X=imputer_test_df)

# Convert the output of running the test data through iterative imputer to a dataframe
imputer_test_result_df = pd.DataFrame(imputer_test_result, 
                                      columns=imputer.get_feature_names_out())
test_encoded_df["trav_sp_imputed"] = imputer_test_result_df["trav_sp"]

In [ ]:
# Remove maxsev_imname from all dataframes
df.drop(columns="maxsev_imname", inplace=True)

# Dropping the Non-Imputed versions of travel speed, now that imputation has been performed
train_imputed_df = train_encoded_df.drop(columns=["trav_sp", "maxsev_imname"])
test_imputed_df = test_encoded_df.drop(columns=["trav_sp", "maxsev_imname"])

#### Scaling travel speed

In [ ]:
speed_scaler = StandardScaler()

# Fit and transform the scaler on the training set
train_imputed_df["trav_sp_scaled"] = speed_scaler.fit_transform(train_imputed_df["trav_sp_imputed"].to_numpy().reshape(-1,1))

# Only transform on the test set (no data leakage)
test_imputed_df["trav_sp_scaled"] = speed_scaler.transform(test_imputed_df["trav_sp_imputed"].to_numpy().reshape(-1,1))

# Use these dataframes moving forward
# The final, fully preprocessed dataframes (all features encoded and imputed as needed)
# Using "pp" to stand for preprocessed
train_pp_df = train_imputed_df.drop(columns="trav_sp_imputed")
test_pp_df = test_imputed_df.drop(columns="trav_sp_imputed")

### Visualize Variable Representation

In [ ]:
pd.set_option('display.max_rows', None)
table = [
    ['age_im', 'int'],
    ['maxsev_binned', 'object'],
    ['mod_yearname_x', 'int'],
    ['ordinal_encoder__urbanicityname', 'ordinal encoded'],
    ['ordinal_encoder__alchl_imname', 'ordinal encoded'],
    ['ordinal_encoder__reljct1_imname', 'ordinal encoded'],
    ['ordinal_encoder__int_hwyname', 'ordinal encoded'],
    ['ordinal_encoder__sex_imname', 'ordinal encoded'],
    ['one_hot_encoder__hour_binned_Morning', 'one hot encoded'],
    ['one_hot_encoder__hour_binned_Night', 'one hot encoded'],
    ['one_hot_encoder__hour_binned_Afternoon', 'one hot encoded'],
    ['one_hot_encoder__hour_binned_Evening', 'one hot encoded'],
    ['one_hot_encoder__intersection_binned_Yes', 'one hot encoded'],
    ['one_hot_encoder__intersection_binned_No', 'one hot encoded'],
    ['one_hot_encoder__intersection_binned_Other','one hot encoded'],
    ['one_hot_encoder__lgtcon_imname_Daylight', 'one hot encoded'],
    ['one_hot_encoder__lgtcon_imname_Dark - Not Lighted', 'one hot encoded'],
    ['one_hot_encoder__lgtcon_imname_Dawn', 'one hot encoded'],
    ['one_hot_encoder__lgtcon_imname_Dark - Lighted', 'one hot encoded'],
    ['one_hot_encoder__lgtcon_imname_Dark - Unknown Lighting', 'one hot encoded'],
    ['one_hot_encoder__lgtcon_imname_Dusk', 'one hot encoded'],
    ['one_hot_encoder__lgtcon_imname_Other', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Going Straight', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Negotiating a Curve', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Stopped in Roadway', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Turning Right', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Turning Left', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Starting in Road', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Changing Lanes', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Decelerating in Road', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Passing or Overtaking Another Vehicle', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Backing Up (other than for Parking Position)', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Making a U-turn', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Accelerating in Road', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Disabled or "Parked" in Travel lane', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Leaving a Parking Position', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Merging', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Entering a Parking Position', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Other(specify:)', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_Successful Avoidance Maneuver to a Previous Critical Event', 'one hot encoded'],
    ['one_hot_encoder__pcrash1_imname_No Driver Present / Unknown if Driver Present', 'one hot encoded'],
    ['one_hot_encoder__regionname_West' 'one_hot_encoder__regionname_South', 'one hot encoded'],
    ['one_hot_encoder__regionname_Northeast', 'one hot encoded'],
    ['one_hot_encoder__regionname_Midwest', 'one hot encoded'],
    ['one_hot_encoder__restraint_binned_Full', 'one hot encoded'],
    ['one_hot_encoder__restraint_binned_Other', 'one hot encoded'],
    ['one_hot_encoder__restraint_binned_None', 'one hot encoded'],
    ['one_hot_encoder__restraint_binned_Minimal', 'one hot encoded'],
    ['one_hot_encoder__vtrafconname_Traffic control signal(on colors) not known whether or not Pedestrian Signal', 'one hot encoded'],
    ['one_hot_encoder__vtrafconname_No Controls', 'one hot encoded'],
    ['one_hot_encoder__vtrafconname_Warning Sign', 'one hot encoded'],
    ['one_hot_encoder__vtrafconname_NR/UNK/OTH', 'one hot encoded'],
    ['one_hot_encoder__vtrafconname_Stop Sign', 'one hot encoded'],
    ['one_hot_encoder__vtrafconname_Traffic control signal (on colors) without Pedestrian Signal', 'one hot encoded'],
    ['one_hot_encoder__vtrafconname_Traffic control signal (on colors) with Pedestrian Signal', 'one hot encoded'],
    ['one_hot_encoder__vtrafconname_Railway Crossing Device', 'one hot encoded'],
    ['one_hot_encoder__vtrafconname_Yield Sign', 'one hot encoded'],
    ['one_hot_encoder__vtrafconname_Flashing Traffic Control Signal', 'one hot encoded'],
    ['one_hot_encoder__vtrafconname_Person', 'one hot encoded'],
    ['one_hot_encoder__vtrafconname_School Zone Sign/Device', 'one hot encoded'],
    ['one_hot_encoder__vtrafconname_Lane Use Control Signal', 'one hot encoded'],
    ['one_hot_encoder__vtrafwayname_Two-Way, Not Divided', 'one hot encoded'],
    ['one_hot_encoder__vtrafwayname_Two-Way,  Divided, Positive  Median Barrier', 'one hot encoded'],
    ['one_hot_encoder__vtrafwayname_NR/UNK', 'one hot encoded'],
    ['one_hot_encoder__vtrafwayname_Entrance/Exit Ramp', 'one hot encoded'],
    ['one_hot_encoder__vtrafwayname_Two-Way, Divided, Unprotected Median', 'one hot encoded'],
    ['one_hot_encoder__vtrafwayname_One-Way Trafficway', 'one hot encoded'],
    ['one_hot_encoder__vtrafwayname_Two-Way, Not Divided With a Continuous Left-Turn Lane', 'one hot encoded'],
    ['one_hot_encoder__vtrafwayname_Non-Trafficway or Driveway Access', 'one hot encoded'],
    ['one_hot_encoder__weather_binned_Not Clear', 'one hot encoded'],
    ['one_hot_encoder__weather_binned_Clear', 'one hot encoded'],
    ['one_hot_encoder__weather_binned_Wintery', 'one hot encoded'],
    ['one_hot_encoder__weather_binned_Other', 'one hot encoded'],
    ['one_hot_encoder__weather_binned_Rain', 'one hot encoded'],
    ['one_hot_encoder__weather_binned_Windy', 'one hot encoded'],
    ['one_hot_encoder__wkdy_imname_Wednesday', 'one hot encoded'],
    ['one_hot_encoder__wkdy_imname_Friday', 'one hot encoded'],
    ['one_hot_encoder__wkdy_imname_Thursday', 'one hot encoded'],
    ['one_hot_encoder__wkdy_imname_Saturday', 'one hot encoded'],
    ['one_hot_encoder__wkdy_imname_Sunday', 'one hot encoded'],
    ['one_hot_encoder__wkdy_imname_Monday', 'one hot encoded'],
    ['one_hot_encoder__wkdy_imname_Tuesday', 'one hot encoded'],
    ['one_hot_encoder__vnum_lanname_Five lanes', 'one hot encoded'],
    ['one_hot_encoder__vnum_lanname_Two lanes', 'one hot encoded'],
    ['one_hot_encoder__vnum_lanname_NR/UNK', 'one hot encoded'],
    ['one_hot_encoder__vnum_lanname_Three lanes', 'one hot encoded'],
    ['one_hot_encoder__vnum_lanname_Four lanes', 'one hot encoded'],
    ['one_hot_encoder__vnum_lanname_Six lanes', 'one hot encoded'],
    ['one_hot_encoder__vnum_lanname_Non-Trafficway or Driveway Access', 'one hot encoded'],
    ['one_hot_encoder__vnum_lanname_Seven or more lanes', 'one hot encoded'],
    ['one_hot_encoder__vnum_lanname_One lane', 'one hot encoded'],
    ['one_hot_encoder__vsurcondname_Snow', 'one hot encoded'],
    ['one_hot_encoder__vsurcondname_Dry', 'one hot encoded'],
    ['one_hot_encoder__vsurcondname_NR/UNK/OTH', 'one hot encoded'],
    ['one_hot_encoder__vsurcondname_Wet', 'one hot encoded'],
    ['one_hot_encoder__vsurcondname_Non-Trafficway or Driveway Access', 'one hot encoded'],
    ['one_hot_encoder__vsurcondname_Ice/Frost', 'one hot encoded'],
    ['one_hot_encoder__vsurcondname_Water (Standing or Moving)', 'one hot encoded'],
    ['one_hot_encoder__vsurcondname_Slush', 'one hot encoded'],
    ['one_hot_encoder__vsurcondname_Sand', 'one hot encoded'],
    ['one_hot_encoder__vsurcondname_Mud, Dirt or Gravel', 'one hot encoded'],
    ['one_hot_encoder__vsurcondname_Oil', 'one hot encoded'],
    ['one_hot_encoder__speeding_status_unknown', 'one hot encoded'],
    ['one_hot_encoder__speeding_status_speeding', 'one hot encoded'],
    ['one_hot_encoder__speeding_status_not speeding', 'one hot encoded'],
    ['one_hot_encoder__body_type_binned_1', 'one hot encoded'],
    ['one_hot_encoder__body_type_binned_2', 'one hot encoded'],
    ['one_hot_encoder__body_type_binned_6', 'one hot encoded'],
    ['one_hot_encoder__body_type_binned_3', 'one hot encoded'],
    ['one_hot_encoder__body_type_binned_9', 'one hot encoded'],
    ['one_hot_encoder__body_type_binned_12', 'one hot encoded'],
    ['one_hot_encoder__body_type_binned_7', 'one hot encoded'],
    ['one_hot_encoder__body_type_binned_10', 'one hot encoded'],
    ['one_hot_encoder__body_type_binned_4', 'one hot encoded'],
    ['one_hot_encoder__body_type_binned_11', 'one hot encoded'],
    ['one_hot_encoder__body_type_binned_5', 'one hot encoded'],
    ['one_hot_encoder__body_type_binned_8', 'one hot encoded'],
    ['trav_sp_scaled', 'float']
]
table = pd.DataFrame(table, columns=['Variable', 'Representation'])
table

,Variable,Representation
0,age_im,int
1,maxsev_binned,object
2,mod_yearname_x,int
3,ordinal_encoder__urbanicityname,ordinal encoded
4,ordinal_encoder__alchl_imname,ordinal encoded
5,ordinal_encoder__reljct1_imname,ordinal encoded
6,ordinal_encoder__int_hwyname,ordinal encoded
7,ordinal_encoder__sex_imname,ordinal encoded
8,one_hot_encoder__hour_binned_Morning,one hot encoded
9,one_hot_encoder__hour_binned_Night,one hot encoded


In [ ]:
pd.reset_option("display.max_rows")

In [ ]:
pd.set_option('max_colwidth', None)
data = [
    ['restraint_binned', 'Object/Categorical', 'Restraint equipment used by driver', 'Full, Minimal, None, Other (one hot encoded)'],
    ['pcrash1_imname', 'Object/Categorical', "Driver's pre-event movenent prior to recognition of critical event", '19 (one hot encoded)'],
    ['speeding_status', 'Object/Categorical', 'Whether driver was above or below speeding limit', 'Speeding, Not Speeding, Unknown (one hot encoded)'],
    ['hour_binned', 'Object/Categorical', 'Time of accident', 'Morning (6am-noon), afternoon (noon-6pm), evening (6pm-midnight), night (midnight-6am) (one hot encoded)'],
    ['body_type_binned', 'Object/Categorical', 'Body type of vehicle', '12 (one hot encoded)'],
    ['weather_binned', 'Object/Categorical', 'Weather at time of accident', 'Clear, Not Clear, Rain, Wintery, Windy, Other (one hot encoded)'],
    ['trav_sp_imputed', 'Int/Numerical', 'Traveling speed of the car', '-'],
    ['intersection_binned', 'Object/Categorical', 'Whether accident took place at an intersection', 'Yes,  No, Other (one hot encoded'],
    ['maxsev_binned', 'Object/Categorical', 'Severity of injury sustained by the driver', 'Fatal, Not Fatal']
]
data = pd.DataFrame(data, columns=[['Feature Name', 'Datatype', 'Description', 'Levels, if applicable']])
data

,Feature Name,Datatype,Description,"Levels, if applicable"
0,restraint_binned,Object/Categorical,Restraint equipment used by driver,"Full, Minimal, None, Other (one hot encoded)"
1,pcrash1_imname,Object/Categorical,Driver's pre-event movenent prior to recognition of critical event,19 (one hot encoded)
2,speeding_status,Object/Categorical,Whether driver was above or below speeding limit,"Speeding, Not Speeding, Unknown (one hot encoded)"
3,hour_binned,Object/Categorical,Time of accident,"Morning (6am-noon), afternoon (noon-6pm), evening (6pm-midnight), night (midnight-6am) (one hot encoded)"
4,body_type_binned,Object/Categorical,Body type of vehicle,12 (one hot encoded)
5,weather_binned,Object/Categorical,Weather at time of accident,"Clear, Not Clear, Rain, Wintery, Windy, Other (one hot encoded)"
6,trav_sp_imputed,Int/Numerical,Traveling speed of the car,-
7,intersection_binned,Object/Categorical,Whether accident took place at an intersection,"Yes, No, Other (one hot encoded"
8,maxsev_binned,Object/Categorical,Severity of injury sustained by the driver,"Fatal, Not Fatal"


In [ ]:
pd.reset_option("display.max_columns")
pd.reset_option("display.max_rows")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50535 entries, 0 to 54471
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age_im               50535 non-null  int64  
 1   alchl_imname         50535 non-null  object 
 2   body_type_binned     50535 non-null  object 
 3   hour_binned          50535 non-null  object 
 4   int_hwyname          50535 non-null  object 
 5   intersection_binned  50535 non-null  object 
 6   lgtcon_imname        50535 non-null  object 
 7   maxsev_binned        50535 non-null  object 
 8   mod_yearname_x       50535 non-null  int64  
 9   pcrash1_imname       50535 non-null  object 
 10  regionname           50535 non-null  object 
 11  reljct1_imname       50535 non-null  object 
 12  restraint_binned     50535 non-null  object 
 13  sex_imname           50535 non-null  object 
 14  speeding_status      50535 non-null  object 
 15  trav_sp              50535 non-null 

In [ ]:
display(df)

,age_im,alchl_imname,body_type_binned,hour_binned,int_hwyname,intersection_binned,lgtcon_imname,maxsev_binned,mod_yearname_x,pcrash1_imname,...,sex_imname,speeding_status,trav_sp,urbanicityname,vnum_lanname,vsurcondname,vtrafconname,vtrafwayname,weather_binned,wkdy_imname
0,61,No Alcohol Involved,1,Morning,No,Yes,Daylight,Not Fatal,2018,Going Straight,...,Female,unknown,25.0,Rural,Five lanes,Snow,Traffic control signal(on colors) not known whether or not Pedestrian Signal,"Two-Way, Not Divided",Not Clear,Wednesday
1,23,No Alcohol Involved,1,Night,No,No,Dark - Not Lighted,Not Fatal,2013,Going Straight,...,Male,speeding,45.0,Urban,Two lanes,Dry,No Controls,"Two-Way, Not Divided",Clear,Wednesday
3,20,No Alcohol Involved,2,Afternoon,Yes,No,Daylight,Not Fatal,2015,Going Straight,...,Male,not speeding,65.0,Rural,Two lanes,Snow,No Controls,"Two-Way, Divided, Positive Median Barrier",Wintery,Wednesday
4,23,No Alcohol Involved,6,Evening,No,No,Dark - Not Lighted,Not Fatal,2004,Negotiating a Curve,...,Male,unknown,998.0,Rural,NR/UNK,Snow,Warning Sign,NR/UNK,Wintery,Wednesday
5,57,No Alcohol Involved,3,Evening,No,No,Dark - Not Lighted,Not Fatal,2003,Going Straight,...,Female,unknown,998.0,Rural,NR/UNK,Dry,No Controls,NR/UNK,Clear,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54467,61,No Alcohol Involved,2,Afternoon,No,No,Daylight,Not Fatal,2017,Going Straight,...,Female,unknown,998.0,Rural,Two lanes,Dry,No Controls,"Two-Way, Not Divided",Clear,Monday
54468,70,No Alcohol Involved,3,Evening,No,No,Daylight,Not Fatal,2019,Going Straight,...,Male,unknown,998.0,Rural,Two lanes,Dry,No Controls,"Two-Way, Not Divided",Not Clear,Wednesday
54469,59,No Alcohol Involved,6,Evening,No,No,Dark - Not Lighted,Not Fatal,1999,Negotiating a Curve,...,Male,unknown,998.0,Urban,Two lanes,Dry,NR/UNK/OTH,"Two-Way, Not Divided",Clear,Thursday
54470,67,No Alcohol Involved,3,Morning,No,No,Daylight,Not Fatal,2006,Going Straight,...,Male,unknown,998.0,Urban,Two lanes,Dry,NR/UNK/OTH,"Two-Way, Not Divided",Clear,Tuesday


In [ ]:
# After preprocessing, final dataframe consisted of 50535 observations, 119 columns
# Here is the dimension of the 80% TRAIN DATAFRAME (40428 rows × 119 columns)
# Due to the extensive columns after hot encoding, 
# please refer to Visualize Variable Representation above for the full list of columns
# Note 119 entries in this call corresponds to 119 columns in the display() call above
train_pp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40428 entries, 0 to 40427
Columns: 121 entries, age_im to Not Fatal
dtypes: float64(120), object(1)
memory usage: 37.3+ MB


In [ ]:
# After preprocessing, final dataframe consisted of 50535 observations, 119 columns
# Here is the dimension of the 80% TRAIN DATAFRAME (40428 rows × 119 columns)
display(train_pp_df)

,age_im,maxsev_binned,mod_yearname_x,ordinal_encoder__urbanicityname,ordinal_encoder__alchl_imname,ordinal_encoder__reljct1_imname,ordinal_encoder__int_hwyname,ordinal_encoder__sex_imname,one_hot_encoder__hour_binned_Morning,one_hot_encoder__hour_binned_Night,...,one_hot_encoder__body_type_binned_12,one_hot_encoder__body_type_binned_7,one_hot_encoder__body_type_binned_10,one_hot_encoder__body_type_binned_4,one_hot_encoder__body_type_binned_11,one_hot_encoder__body_type_binned_5,one_hot_encoder__body_type_binned_8,trav_sp_scaled,Fatal,Not Fatal
0,2.530925,Not Fatal,0.625929,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.573139,0.0,1.0
1,0.194944,Not Fatal,-2.002092,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040888,0.0,1.0
2,1.961173,Not Fatal,0.771930,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.929556,0.0,1.0
3,0.365870,Not Fatal,1.063933,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.775295,0.0,1.0
4,-0.602708,Not Fatal,-0.396079,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.272600,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40423,0.536795,Not Fatal,0.479928,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.376373,0.0,1.0
40424,0.479820,Not Fatal,1.355935,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.573139,0.0,1.0
40425,-0.773633,Fatal,-1.418087,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.599395,1.0,0.0
40426,-0.032956,Not Fatal,1.063933,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.929556,0.0,1.0


In [ ]:
# After preprocessing, final dataframe consisted of 50535 observations, 119 columns
# Here is the dimension of the 20% TEST DATAFRAME (10107 rows × 119 columns)
# Due to the extensive columns after hot encoding, 
# please refer to Visualize Variable Representation above for the full list of columns
# Note 119 entries in this call corresponds to 119 columns in the display() call above
test_pp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10107 entries, 0 to 10106
Columns: 119 entries, age_im to trav_sp_scaled
dtypes: float64(118), object(1)
memory usage: 9.2+ MB


In [ ]:
# After preprocessing, final dataframe consisted of 50535 observations, 119 columns
# Here is the dimension of the 20% TEST DATAFRAME (10107 rows × 119 columns)
display(test_pp_df)

,age_im,maxsev_binned,mod_yearname_x,ordinal_encoder__urbanicityname,ordinal_encoder__alchl_imname,ordinal_encoder__reljct1_imname,ordinal_encoder__int_hwyname,ordinal_encoder__sex_imname,one_hot_encoder__hour_binned_Morning,one_hot_encoder__hour_binned_Night,...,one_hot_encoder__body_type_binned_3,one_hot_encoder__body_type_binned_9,one_hot_encoder__body_type_binned_12,one_hot_encoder__body_type_binned_7,one_hot_encoder__body_type_binned_10,one_hot_encoder__body_type_binned_4,one_hot_encoder__body_type_binned_11,one_hot_encoder__body_type_binned_5,one_hot_encoder__body_type_binned_8,trav_sp_scaled
0,-0.032956,Not Fatal,-0.396079,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.376716
1,1.334447,Not Fatal,1.063933,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.929556
2,0.878646,Not Fatal,-0.834082,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.530635
3,-1.001534,Not Fatal,0.917932,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.925151
4,0.422845,Not Fatal,1.063933,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.629017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10102,-0.716658,Fatal,-0.834082,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.152092
10103,-0.602708,Not Fatal,1.063933,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.622488
10104,2.758825,Fatal,0.771930,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.923350
10105,0.422845,Not Fatal,-0.396079,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.272600


# Data Understanding 2
- Visualize the any important attributes appropriately. 
- Important: Provide an interpretation for any charts or graphs.  
(10)

# Model and Evaluation 1
- Train and adjust parameters  
(10)

# Model and Evaluation 2
- Evaluate and compare  
(10)

# Model and Evaluation 3
- Visualize results  
(10)

# Model and Evaluation 4
- Summarize the ramifications  
(20)

# Deployment
- Be critical of your performance and tell the reader how you current model might be usable by other parties. 
- Did you achieve your goals? If not, can you reign in the utility of your modeling? 
- How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)? 
- How would your deploy your model for interested parties? 
- What other data should be collected? 
- How often would the model need to be updated, etc.?  
(10)

# Exceptional Work
- You have free reign to provide additional analyses or combine analyses.  
(10)

# Appendix
Data dictionary referenced from [CRSS Analytical Users Manual](https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/813236)